In [4]:
from google.colab import drive
drive.mount('/content/drive')
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import h5py
import scipy
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
folder_path = "content/drive/MyDrive/Project/hdf_files"

class GaussianLayer(nn.Module):
    def __init__(self, kernel_size=250, sigma=250):
        super(GaussianLayer, self).__init__()
        self.kernel_size = kernel_size
        self.sigma = sigma

        self.seq = nn.Sequential(
            nn.ReflectionPad2d(kernel_size // 2),
            nn.Conv2d(1, 1, kernel_size, stride=1, padding=0, bias=False)
        )

        self.weights_init()

    def forward(self, x):
        return self.seq(x)

    def weights_init(self):
        n = np.zeros((self.kernel_size, self.kernel_size))
        n[self.kernel_size // 2, self.kernel_size // 2] = 1
        k = scipy.ndimage.gaussian_filter(n, sigma=self.sigma)

        for name, f in self.named_parameters():
            f.data.copy_(torch.from_numpy(k).float().unsqueeze(0).unsqueeze(0))


for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    with h5py.File(file_path, 'r') as f:
        image_data = f['image_data'][:]
        image_tensor = torch.tensor(image_data, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Forme (1, 1, H, W)
        kernel = GaussianLayer(250, sigma=250)
        blurred_image_tensor = F.conv2d(image_tensor, kernel, padding=250//2)